In [17]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [18]:
tomato = pd.read_csv("C:/Users/Darui Yen/OneDrive//桌面/GWAS_pimpinellifolium_list.csv")
tomato["latitude"] = pd.Series()
tomato["longitude"] = pd.Series()
tomato

C:\Users\Darui Yen\Anaconda3\envs\superconda\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\Darui Yen\Anaconda3\envs\superconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


,accession,species,#LA,#PI,AVRDC_code,source,origin,coordinate,latitude,longitude
0,2013A01594,S. pimpinellifolium,LA2181,NaN,VI037972,tainan district agricultural and extension sta...,Peru,NaN,NaN,NaN
1,2017A01921,S. pimpinellifolium,NaN,PI 634844,NaN,tainan district agricultural and extension sta...,United States,NaN,NaN,NaN
2,2017A01922,S. pimpinellifolium,LA2348,PI 79532,NaN,tainan district agricultural and extension sta...,Peru,NaN,NaN,NaN
3,2017A01924,S. pimpinellifolium,NaN,PI124039,NaN,tainan district agricultural and extension sta...,Peru,NaN,NaN,NaN
4,2017A01925,S. pimpinellifolium,LA2934,PI126430,NaN,tainan district agricultural and extension sta...,Peru,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
230,LA2647,S. pimpinellifolium,LA2647,NaN,NaN,213 lab,NaN,NaN,NaN,NaN
231,LA2653,S. pimpinellifolium,LA2653,NaN,NaN,213 lab,NaN,NaN,NaN,NaN
232,LA2655,S. pimpinellifolium,LA2655,NaN,NaN,213 lab,NaN,NaN,NaN,NaN
233,LA2656,S. pimpinellifolium,LA2656,NaN,NaN,213 lab,NaN,NaN,NaN,NaN


In [19]:
#lab來源的品系無座標和國家，要從TGRC補
index_nocountry = list()
for i in range(tomato.shape[0]) :
    if type(tomato["origin"][i]) != str :
        index_nocountry.append(i)

#農改場來源的品系座標來自USDA，因此要補上TGRC的資料，但要避免和lab來源重複的21個品系
index_noduplicate = list()
index_duplicate = list()

#去除空格 & 區分無重複品系
edit_accession = [x.strip() for x in tomato["accession"]]

for i in range(tomato.shape[0]) :
    #判斷是否有值
    if type(tomato["#LA"][i]) != float :
        #去除中間空格
        if len(tomato["#LA"][i].strip()) > 1 :
            tomato["#LA"][i] = "".join(tomato["#LA"][i].split())
        #判斷是否重複   
        if tomato["#LA"][i].strip() in edit_accession :
            index_duplicate.append(i)
        else :
            index_noduplicate.append(i)

C:\Users\Darui Yen\Anaconda3\envs\superconda\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [20]:
#抓TGRC網頁
def crawl_web(TGRC_accession, index) :
    accession = TGRC_accession[index]
    print("Start with accession {}. (index = {})".format(accession, index))
    web = "https://tgrc.ucdavis.edu/Data/Acc/AccDetail.aspx?AccessionNum={}&contains=false".format(accession)
    response = requests.get(web)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup

#抓經緯度
def crawl_coor() :
    location = soup.find("span", id = "lLatLon")
    if len(location) < 5 :
        lati = longi = np.nan
    else :
        lati = location.contents[2].string
        longi = location.contents[4].string
    return lati, longi

#抓採集地
def crawl_country() :
    country = soup.find("span", id = "lCountry")
    if len(country) <2 : 
        nation = np.nan
    else : 
        nation = country.contents[2].string
    return nation

In [21]:
#補農改場來源的位置
for index in index_noduplicate :
    soup = crawl_web(tomato["#LA"], index)
    
    #經緯度
    tomato["latitude"][index], tomato["longitude"][index] = crawl_coor()
print("All accessions are done.")

Start with accession LA2348. (index = 2)


C:\Users\Darui Yen\Anaconda3\envs\superconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Start with accession LA2934. (index = 4)


C:\Users\Darui Yen\Anaconda3\envs\superconda\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Start with accession LA0381. (index = 32)
Start with accession LA0384. (index = 33)
Start with accession LA0397. (index = 34)
Start with accession LA0412. (index = 37)
Start with accession LA0413. (index = 38)
Start with accession LA1993. (index = 54)
Start with accession LA1246. (index = 62)
Start with accession LA1259. (index = 63)
Start with accession LA1260. (index = 64)
Start with accession LA1261. (index = 65)
Start with accession LA1262. (index = 66)
Start with accession LA1263. (index = 67)
Start with accession LA1269. (index = 68)
Start with accession LA1332. (index = 69)
Start with accession LA1342. (index = 71)
Start with accession LA1344. (index = 72)
Start with accession LA1349. (index = 73)
Start with accession LA1355. (index = 74)
Start with accession LA1359. (index = 75)
Start with accession LA1375. (index = 76)
Start with accession LA1279. (index = 78)
Start with accession LA1341. (index = 79)
Start with accession LA1345. (index = 80)
Start with accession LA1370. (inde

In [22]:
#抓lab來源的位置和國家
for index in index_nocountry :
    soup = crawl_web(tomato["accession"], index)
    
    #經緯度
    tomato["latitude"][index], tomato["longitude"][index] = crawl_coor()
    
    #採集地
    tomato["origin"][index] = crawl_country()

print("All accessions are done.")

Start with accession LA0114. (index = 172)
Start with accession LA0373. (index = 173)
Start with accession LA0400. (index = 174)
Start with accession LA0411. (index = 175)
Start with accession LA0442. (index = 176)
Start with accession LA1236. (index = 177)
Start with accession LA1237. (index = 178)
Start with accession LA1245. (index = 179)
Start with accession LA1280. (index = 180)
Start with accession LA1301. (index = 181)
Start with accession LA1335. (index = 182)
Start with accession LA1348. (index = 183)
Start with accession LA1380. (index = 184)
Start with accession LA1381. (index = 185)
Start with accession LA1382. (index = 186)
Start with accession LA1466. (index = 187)
Start with accession LA1469. (index = 188)
Start with accession LA1478. (index = 189)
Start with accession LA1514. (index = 190)
Start with accession LA1521. (index = 191)
Start with accession LA1547. (index = 192)
Start with accession LA1578. (index = 193)
Start with accession LA1580. (index = 194)
Start with 

In [23]:
#修剪TGRC爬到的座標
for direction in ["latitude", "longitude"] :
    for i in range(tomato.shape[0]) :
        if type(tomato[direction][i]) != float :
            if len(tomato[direction][i]) < 30 :
                tomato[direction][i] = tomato[direction][i].strip()
            else :
                tomato[direction][i] = tomato[direction][i].split(":")[1].strip()

In [25]:
#修剪USDA整理的座標
for i in range(tomato.shape[0]) :
    if type(tomato["coordinate"][i]) != float :
        coor = tomato["coordinate"][i].split(",")
        tomato["latitude"][i], tomato["longitude"][i] = coor[0].strip(), coor[1].strip()
        
tomato = tomato.drop("coordinate", axis = 1)

In [27]:
tomato.to_csv("C:/Users/Darui Yen/OneDrive/桌面/crawler_pimpinellifolium_list.csv", index = False)

In [28]:
len(index_duplicate)
#64 from lab + 21 which duplicated 

85

In [29]:
len(index_noduplicate)

47

In [30]:
len(index_nocountry)
#exactly accessions from lab

63

In [31]:
tomato

,accession,species,#LA,#PI,AVRDC_code,source,origin,latitude,longitude
0,2013A01594,S. pimpinellifolium,LA2181,NaN,VI037972,tainan district agricultural and extension sta...,Peru,NaN,NaN
1,2017A01921,S. pimpinellifolium,NaN,PI 634844,NaN,tainan district agricultural and extension sta...,United States,NaN,NaN
2,2017A01922,S. pimpinellifolium,LA2348,PI 79532,NaN,tainan district agricultural and extension sta...,Peru,NaN,NaN
3,2017A01924,S. pimpinellifolium,NaN,PI124039,NaN,tainan district agricultural and extension sta...,Peru,NaN,NaN
4,2017A01925,S. pimpinellifolium,LA2934,PI126430,NaN,tainan district agricultural and extension sta...,Peru,-11.854444,-77.037500
...,...,...,...,...,...,...,...,...,...
230,LA2647,S. pimpinellifolium,LA2647,NaN,NaN,213 lab,Peru,-5.175000,-79.983333
231,LA2653,S. pimpinellifolium,LA2653,NaN,NaN,213 lab,Peru,-4.750000,-80.583333
232,LA2655,S. pimpinellifolium,LA2655,NaN,NaN,213 lab,Peru,-4.908333,-80.825000
233,LA2656,S. pimpinellifolium,LA2656,NaN,NaN,213 lab,Peru,-3.800000,-80.700000
